# Exploratory Data Analysis and Initial Model Creation

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import scipy.stats as stats
import modeling
import data_viz
import data_cleaning
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
%matplotlib inline
sns.set(font_scale=1.5)


In [36]:
df = pd.read_csv('datasets/train_cleaned.csv')
df_test = pd.read_csv('datasets/test_cleaned.csv')
model_df = pd.read_csv('datasets/models.csv')
modeling.test_columns(df=df, df_test=df_test)
%store -r features5

In [37]:
df.shape

(2034, 87)

In [38]:
df_test.shape

(879, 206)

# EDA for Potential Categorical Features

In [39]:
mean_price = df.saleprice.mean()

def greater_than_mean(columns,data=df,mean_price=mean_price):
    for column in columns:
        print(df.groupby(by = column
              ).mean()['saleprice'].T.sort_values(ascending=False) >= mean_price)
        print('------------------------')
potential_cats = ['ms_subclass',
                  'lot_config',
                  'neighborhood',
                  'condition_2',
                  'house_style',
                  'roof_matl',
                  'exterior_1st',
                  'exterior_2nd',
                  'bsmtfin_type_1',
                  'central_air',
                  'garage_type'
                 ]

From this you can you get the info of what items correlate with a higher than the average sale price. Don't run it!

In [ ]:
data_viz.graph_greater_than_mean(potential_cats,
                       df,
                       mean_price+mean_price*.10,
                       target='saleprice')
plt.savefig('resources/cats_salemean.png')

## 1 Hot Creating Dummy Variables for the existing non-numeric variables

In [ ]:
dummy_cols = potential_cats

In [ ]:
df = pd.get_dummies(df, columns=dummy_cols, drop_first=True)
df.shape

In [ ]:
df_test = pd.get_dummies(df_test, columns=dummy_cols, drop_first=True)
df_test.shape

# Export your test data here!

In [ ]:
data_cleaning.snake_df(df)
data_cleaning.snake_df(df_test)
modeling.test_columns(df, df_test)
df.to_csv('datasets/train_cleaned_dummied.csv', index=False)
df_test.to_csv('datasets/test_cleaned_dummied.csv', index=False)

Reimport it in case you start over again.

In [ ]:
df = pd.read_csv('datasets/train_cleaned_dummied.csv')
df_test = pd.read_csv('datasets/test_cleaned_dummied.csv')
model_df = pd.read_csv('datasets/models.csv')

In [ ]:
df.shape

In [ ]:
df_test.shape

## Examining our encoded variables
Start over again with the correlation map

In [ ]:
encoded = ['ms_subclass_1-1/2_story_pud_-_all_ages',
       'ms_subclass_1-1/2_story_unfinished_all_ages',
       'ms_subclass_1-story_1945_older',
       'ms_subclass_1-story_1946_newer_all_styles',
       'ms_subclass_1-story_finished_attic_all_ages',
       'ms_subclass_1-story_pud_(planned_unit_development)_-_1946_newer',
       'ms_subclass_2_family_conversion_-_all_styles_and_ages',
       'ms_subclass_2-1/2_story_all_ages',
       'ms_subclass_2-story_1945_older', 'ms_subclass_2-story_1946_newer',
       'ms_subclass_2-story_pud_-_1946_newer',
       'ms_subclass_duplex_-_all_styles_and_ages',
       'ms_subclass_pud_-_multilevel_-_incl_split_lev/foyer',
       'ms_subclass_split_foyer', 'ms_subclass_split_or_multi-level',
       'lot_config_culdsac', 'lot_config_fr2', 'lot_config_fr3',
       'lot_config_inside', 'neighborhood_blueste', 'neighborhood_brdale',
       'neighborhood_brkside', 'neighborhood_clearcr',
       'neighborhood_collgcr', 'neighborhood_crawfor',
       'neighborhood_edwards', 'neighborhood_gilbert',
       'neighborhood_greens', 'neighborhood_grnhill',
       'neighborhood_idotrr', 'neighborhood_landmrk',
       'neighborhood_meadowv', 'neighborhood_mitchel',
       'neighborhood_names', 'neighborhood_noridge',
       'neighborhood_npkvill', 'neighborhood_nridght',
       'neighborhood_nwames', 'neighborhood_oldtown',
       'neighborhood_sawyer', 'neighborhood_sawyerw',
       'neighborhood_somerst', 'neighborhood_stonebr',
       'neighborhood_swisu', 'neighborhood_timber',
       'neighborhood_veenker', 'condition_2_feedr', 'condition_2_norm',
       'condition_2_posa', 'condition_2_posn', 'condition_2_rrae',
       'condition_2_rran', 'condition_2_rrnn', 'house_style_1.5unf',
       'house_style_1story', 'house_style_2.5fin', 'house_style_2.5unf',
       'house_style_2story', 'house_style_sfoyer', 'house_style_slvl',
       'roof_matl_membran', 'roof_matl_tar&grv', 'roof_matl_wdshake',
       'roof_matl_wdshngl', 'exterior_1st_asphshn',
       'exterior_1st_brkcomm', 'exterior_1st_brkface',
       'exterior_1st_cblock', 'exterior_1st_cemntbd',
       'exterior_1st_hdboard', 'exterior_1st_imstucc',
       'exterior_1st_metalsd', 'exterior_1st_plywood',
       'exterior_1st_stone', 'exterior_1st_stucco',
       'exterior_1st_vinylsd', 'exterior_1st_wd_sdng',
       'exterior_1st_wdshing', 'exterior_2nd_asphshn',
       'exterior_2nd_brk_cmn', 'exterior_2nd_brkface',
       'exterior_2nd_cblock', 'exterior_2nd_cmentbd',
       'exterior_2nd_hdboard', 'exterior_2nd_imstucc',
       'exterior_2nd_metalsd', 'exterior_2nd_plywood',
       'exterior_2nd_stone', 'exterior_2nd_stucco',
       'exterior_2nd_vinylsd', 'exterior_2nd_wd_sdng',
       'exterior_2nd_wd_shng', 'bsmtfin_type_1_blq', 'bsmtfin_type_1_glq',
       'bsmtfin_type_1_lwq', 'bsmtfin_type_1_na', 'bsmtfin_type_1_rec',
       'bsmtfin_type_1_unf', 'central_air_y', 'garage_type_attchd',
       'garage_type_basment', 'garage_type_builtin',
       'garage_type_carport', 'garage_type_detchd', 'garage_type_na','logsaleprice']

In [ ]:
np.abs(df[encoded].corr()[['logsaleprice']].sort_values(by='logsaleprice', ascending=False)[:10]) >= .3

In [ ]:
features6 = ['bsmtfin_sf_1',
'heating_qc',
'fireplace_qu',
'year_built',
'bsmt_qual',
'1st_flr_sf',
'garage_area',
'kitchen_qual',
'gr_liv_area',
'exter_qual',
'overall_qual',
'porches',
'baths',
'open_porch_sf',
'full_bath',
'lot',
'lot_area',
'overall_cond',
'bsmtfin_sf_1',
'2nd_flr_sf',
'half_bath',
'total_bsmt_sf',
'fireplaces',
'year_remod/add',
'central_air_y',
'neighborhood_nridght',
'neighborhood_stonebr',
'neighborhood_noridge',
]

target = 'logsaleprice'

model_df = modeling.new_test(df = df,
        test_df = df_test,
        features = features6,
        target = target,
         random_state = 1,
         model_type = LinearRegression,
         model_df = model_df,
        scaled=False,
        );
y=df[target]
ols_X = sm.add_constant(df[features6])
model6_ols = sm.OLS(y,ols_X).fit()
model6_ols.summary()

In [ ]:
data_cleaning.snake_df(df)
data_cleaning.snake_df(df_test)
modeling.test_columns(df, df_test)
df.to_csv('datasets/train_cleaned.csv', index=False)
df_test.to_csv('datasets/test_cleaned.csv', index=False)
model_df.to_csv('datasets/models.csv')

### Move on to the last stage: [Algorithmic Parameter Choice & Feature Selection]('4_Algorithmic_feature_parameter_choice.ipynb')